In [ ]:
import numpy as np
import pandas as pd
import os
import sklearn
import rpy2

In [ ]:
# Load data 
antonPath = False
alexPath = True 
if alexPath == True :
    wd = "C:\\Users\\alexa\\Desktop\\FixedIncomeStrat"
else:
    wd = "/Users/antonaleynikov/Desktop/Fixed Income Sentiment strat"
os.chdir(wd)
os.getcwd()

In [ ]:
for path, folders, files in os.walk(wd):
    print("folders:", folders)
    print("files:", files)
    print("path:", path)

In [ ]:
data_path = os.path.join(wd, "Data")
if alexPath == True :
    datafiles_names = os.listdir(wd + "\\Data")
else :
    datafiles_names = os.listdir(wd + "/Data")
    

filenames = [filename.split(".") for filename in datafiles_names[2:]]


In [ ]:
for file in filenames:
    file_name = file[0]
    file_format = file[1]
    file_path = os.path.join(data_path , ".".join(file))
    df = pd.read_excel(file_path, index_col = "Dates")
    if file_format == "xlsx":
        df = pd.read_excel(file_path, index_col = "Dates")
    globals()[file_name] = df 
locals()

In [ ]:
Yields.head()

In [ ]:
Exogen["SPX_EP_YIELD_SPREAD"] = Exogen["SPX_PE_RATIO"].apply(lambda x: 1 / x) - (Forwards["Average"][:-1] / 100).values
del Exogen["SPX_PE_RATIO"]
Exogen["FORWARDS"] = Forwards["Average"][:-1].values
Exogen

In [ ]:
from sklearn.linear_model import LinearRegression as lm 


def get_lam(yields, lam_seq):
    "get lambda given known yields and a searching grid"
    if 0 in lam_seq:
        print("Remove zero from search grid")
    maturities = np.array([1/12, 1/4,1/2 ,1 , 2, 3, 5, 7, 10, 20, 30])
    r2 = pd.DataFrame(0, index=np.arange(yields.shape[0]), columns=lam_seq)
    for j in range(len(lam_seq)):
        lam = lam_seq[j]
        
        # Initialise regressors based on lambda 
        regs = pd.DataFrame()
        regs["level_reg"] = [1 for t in maturities]
        regs["slope_reg"] = [(1 - np.exp(-lam * t))/(lam * t) for t in maturities]
        regs["curv_reg"] = [(1 - np.exp(-lam * t))/(lam * t) - np.exp(-lam*t) for t in maturities]
        
        # Fit yields to regressors
        factors = pd.DataFrame({"level": "", "slope": "", "curvature": ""},index = yields.index)
        for i in range(yields.shape[0]):
            X = regs.values
            y = yields.iloc[i].values
            reg = lm().fit(X, y)
            level, slope ,curv = reg.intercept_, reg.coef_[1], reg.coef_[2]
            rsq = reg.score(X,y)
            factors.iloc[i] = [level, slope, curv]
            r2.iloc[i,j] = rsq
    lam_max = r2.mean().idxmax()
    return lam_max
        
# get_lam(yields, np.linspace(0.1,1,10))

def fit_factors(yields, lam):
    factors = pd.DataFrame({"level": "", "slope": "", "curvature": ""},index = yields.index)
    
    # Initialise regressors based on lambda
    maturities = np.array([1/12, 1/4,1/2 ,1 , 2, 3, 5, 7, 10, 20, 30])
    regs = pd.DataFrame()
    regs["level_reg"] = [1 for t in maturities]
    regs["slope_reg"] = [(1 - np.exp(-lam * t))/(lam * t) for t in maturities]
    regs["curv_reg"] = [(1 - np.exp(-lam * t))/(lam * t) - np.exp(-lam*t) for t in maturities]
    
    for row in range(factors.shape[0]):
        reg = lm().fit(regs.values, yields.iloc[row].values)
        level = reg.intercept_
        slope , curv = reg.coef_[1], reg.coef_[2]
        factors.iloc[row] = [level, slope, curv]
    return factors

# lam = get_lam(yields, np.linspace(0.1,1,10))
# factors = fit_factors(yields, lam)

In [ ]:
lam = get_lam(Yields, np.linspace(0.1,1,10))
factors = fit_factors(Yields, lam)
dfact = factors.diff()[1:]

In [ ]:
# TEST
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
from rpy2.robjects import pandas2ri
utils = importr("utils")
# utils.chooseCRANmirror(ind=1)
# utils.install_packages('rugarch')
# utils.install_packages('forecast')
forecast = importr('forecast')
rugarch = importr('rugarch')
base = importr('base')

# X is a Panda Series that contains the returns I want to fit the model
returns = FloatVector(dfact["slope"].values)   
with (ro.default_converter + pandas2ri.converter).context():
     exo = ro.conversion.get_conversion().py2rpy(Exogen)

ro.r("""  
     f <- function (series, exogen) {
     spec = ugarchspec(
     mean.model=list(armaOrder=auto.arima(series)$arma[1:2], external.regressors = exogen),
     variance.model=list(garchOrder=c(1,1)),
     distribution.model="sged"
     ) 

     fit = tryCatch(
     ugarchfit(
     spec, series, solver = 'hybrid'
     ), error=function(e) e, warning=function(w) w
     )  
     fore = ugarchforecast(fit, n.ahead=1)
     ind = fore@forecast$seriesFor
     ind
     }""")

fcast_armax = ro.globalenv['f']
fore = fcast_armax(returns, base.as_matrix(exo))[0]

fore

In [ ]:
def roll_armax(yields, exogen_nopca, exogen_pca, factor_name, w_len):
    fcast = {}
    s_len = yields.shape[0] - 1
    for t in range(s_len - w_len):
        end = t + w_len
        lam = get_lam(yields[t:end], np.linspace(0.1,0.7,10))
        dfactors = fit_factors(yields[t:end], lam).diff()[1:]
        
        if exogen_pca.any()[0] == True:
            # Standardise
            scaled = StandardScaler().fit_transform(exogen_pca[t:end])

            # PCA transform
            pca = PCA(n_components=1)  # You can choose the number of components you want to keep
            p_comp = pd.Series(pca.fit_transform(scaled)[:,0], index = exogen_nopca[t:end].index)

            dt = pd.concat([exogen_nopca[t:end], p_comp], axis = 1)
        else:
            dt = exogen_nopca[t:end]

        
        series = FloatVector(dfactors[factor_name].values)   
        with (ro.default_converter + pandas2ri.converter).context():
                exo = ro.conversion.get_conversion().py2rpy(dt)

        fore = fcast_armax(series, base.as_matrix(exo))[0]
        fcast[yields.index.values[end]] = fore

    return pd.Series(fcast)

In [ ]:
w_len = 150
num_rows = 3
num_columns = 4

# Create a DataFrame filled with zeros
zeros_df = pd.DataFrame(0, index=range(num_rows), columns=range(num_columns))
zeros_df.any()[0]
slope_fore = roll_armax(Yields, Exogen, zeros_df, "slope", w_len)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(12,5))
dfact["slope"][w_len:].plot(color = 'blue')
slope_fore.plot(color = 'red')

plt.show

In [ ]:
from math import exp
import numpy as np
import pandas as pd


def predict_yield_at_maturity(level, slope, curvature, maturity, lambda_):
    level_part = level
    slope_part = slope * ((1 - exp(-maturity * lambda_)) / (maturity * lambda_))
    curvature_part = curvature * (((1 - exp(-maturity * lambda_)) / (maturity * lambda_)) - exp(-maturity * lambda_))
    yield_prediction = level_part + slope_part + curvature_part
    return yield_prediction


def dollar_level_duration_zerocoupon(face_value, maturity, yield_prediction):
    '''
  Contains information on how a change in the level factor influences the price change of a bond (with a certain maturity)

  Compute the D0 from page 197 in the book
  face_value = F(maturity) -> 0 for all intermediate timesteps until maturity (zero-coupon)
  '''
    return (-maturity * face_value * exp(-maturity * yield_prediction))


def dollar_slope_duration_zerocoupon(face_value, maturity, yield_prediction, lambda_):
    '''
  Contains information on how a change in the slope factor influences the price change of a bond (with a certain maturity)


  Compute the D1 from page 197 in the book
  face_value = F(maturity) -> 0 for all intermediate timesteps until maturity (zero-coupon)
  '''
    slope_fac = (1 - exp(-maturity * lambda_)) / (maturity * lambda_)
    return slope_fac * (-maturity * face_value * exp(-maturity * yield_prediction))


def dollar_curvature_duration_zerocoupon(face_value, maturity, yield_prediction, lambda_):
    '''
  Contains information on how a change in the curvature factor influences the price change of a bond (with a certain maturity)

  Compute the D2 from page 197 in the book
  face_value = F(maturity) -> 0 for all intermediate timesteps until maturity (zero-coupon)
  '''
    curvature_fac = ((1 - exp(-maturity * lambda_)) / (maturity * lambda_)) - exp(-maturity * lambda_)
    return curvature_fac * (-maturity * face_value * exp(-maturity * yield_prediction))


def predict_price_change_due_to_factor(factor_prev, factor_next, factor_dollar_duration):
    '''
  Computes the predicted change in price due to a change in factor

  factor_dollar_duration: for chosen maturity this gives the ratio at which a change in the factor influences the price
  factor_prev: the last obeserved factor (at time t)
  factor_next: the predicted factor (at time t+1)
  '''
    return (factor_next - factor_prev) * factor_dollar_duration


def compute_hedge_terms(maturities_list, face_value, level_prev, slope_prev, curvature_prev, lambda_):
    # def compute_hedge_terms(maturities_list, face_value, level_prev, level_pred, slope_prev, slope_pred, curvature_prev, curvature_pred):
    '''
  maturities_list: a list of the maturities of the bonds we want to trade
  '''
    D0_list = []
    D1_list = []
    D2_list = []

    # computing the dollar durations for the different factors
    for maturity in maturities_list:
        yield_pred = predict_yield_at_maturity(level=level_prev, slope=slope_prev, curvature=curvature_prev,
                                               maturity=maturity, lambda_=lambda_)
        # for a maturity we look at how sensitive the according bond price is to changes in the factor that describes the yield curve
        D0_list.append(
            dollar_level_duration_zerocoupon(face_value=face_value[0], maturity=maturity, yield_prediction=yield_pred))
        D1_list.append(
            dollar_slope_duration_zerocoupon(face_value=face_value[1], maturity=maturity, yield_prediction=yield_pred,
                                             lambda_=lambda_))
        D2_list.append(dollar_curvature_duration_zerocoupon(face_value=face_value[2], maturity=maturity,
                                                            yield_prediction=yield_pred, lambda_=lambda_))

    # predicting price changes for different factors (D0=level, D1=slope, D2=curvature) across multiple maturities (thats why using list)
    # D0_price_changes_list = [(level_pred - level_prev) * D0_at_mat for D0_at_mat in D0_list]
    # D1_price_changes_list = [(slope_pred - slope_prev) * D1_at_mat for D1_at_mat in D1_list]
    # D2_price_changes_list = [(curvature_pred - curvature_prev) * D2_at_mat for D2_at_mat in D2_list]

    return D0_list, D1_list, D2_list


def compute_hedge_weights_basic(D0_list, D1_list, D2_list, wanted_exposure="curvature"):
    '''
  Uses the basic hedging that is used in the original paper.
  Idea: choose the weights in such a way that the exposure to two of the thre factors cancel out across the multiple maturities

  Carefull: If a factor is contained in the hedging equations
  '''

    to_hedge = []

    if wanted_exposure == "level":
        to_hedge = [D0_list, D2_list]
    elif wanted_exposure == "slope":
        to_hedge = [D1_list, D2_list]
    elif wanted_exposure == "curvature":
        to_hedge = [D0_list, D1_list]
    else:
        print("ERROR: Chose a wanted_exposure that does not exist!")

    # Construct the linear system that includes the hedging equations: A*x=b
    A = np.ones((3, 3))
    for i in range(len(to_hedge)):
        A[i, :] = to_hedge[i]
    b = np.array([0, 0, 1])
    b.shape = (3, 1)  # make it a column vector

    # solve the linear system: x = A(-1) * b
    A_inv = np.linalg.inv(A)
    exposure_weights = A_inv.dot(b)

    return exposure_weights


def compute_hedge_weights_basic_df(actual_factors_df, predicted_factor_changes_df, maturities_list, face_value,
                                   lambdas):
    '''
  # New
  Takes a dataframe.
  '''
    predicted_weights = pd.DataFrame()
    D0_list_full = []
    D1_list_full = []
    D2_list_full = []

    for i in range(0, len(predicted_factor_changes_df)):
        # load realized factors from last month (i)
        # level_prev = actual_factors_df.loc[i, 'level']
        # slope_prev = actual_factors_df.loc[i, 'slope']
        # curvature_prev = actual_factors_df.loc[i, 'curvature']

        # compute predicted factors for this month (i) based one (VAR) predicted changes in the factor
        # level_pred = level_prev + predicted_factor_changes_df.loc[i, 'level']
        # slope_pred = slope_prev + predicted_factor_changes_df.loc[i, 'slope']
        # curvature_pred = curvature_prev + predicted_factor_changes_df.loc[i, 'curvature']

        # getting the realized factors of last month
        level_realized = actual_factors_df.loc[i, 'level']
        slope_realized = actual_factors_df.loc[i, 'slope']
        curvature_realized = actual_factors_df.loc[i, 'curvature']
        lambda_realized = lambdas.loc[len(predicted_factor_changes_df) + 1 + i][1]

        # computing the sensitivities of the yield curve (at different maturities) to factor changes
        D0_list, D1_list, D2_list = compute_hedge_terms(maturities_list, face_value, level_realized, slope_realized,
                                                        curvature_realized, lambda_realized)
        D0_list_full.append(D0_list)
        D1_list_full.append(D1_list)
        D2_list_full.append(D2_list)

        # choosing exposure weights s.t. the sensitivity to 2 of the 3 factors are cancelled out and only exposure to one factor is left
        predicted_weights = predicted_weights._append(pd.Series(
        compute_hedge_weights_basic(D0_list, D1_list, D2_list, wanted_exposure).reshape(1, 3).tolist()[0]),
            ignore_index=True)

    return predicted_weights, D0_list_full, D1_list_full, D2_list_full